In [1]:
import keras
import keras.backend as K
from keras.losses import mean_squared_error
import tensorflow as tf
import pandas as pd
import numpy as np
import math
from support_fucntions import parse_train_mass, parse_test_answers

global RNN_model

In [49]:
df = pd.read_csv("dataset/train.csv")
# перевожу ее в массив
train_mass_csv = np.array(df)

# equal_train_mass = []

# count_0, count_1, count_2 = (0,0,0)

# for row in train_mass_csv:
#     match row[10]:
#         case 0:
#             if (count_0 <= 600):
#                 equal_train_mass.append(row)
#                 count_0 += 1
#         case 1:
#             if (count_1 <= 200):
#                 equal_train_mass.append(row)
#                 count_1 += 1
#         case 2:
#             if (count_2 <= 200):
#                 equal_train_mass.append(row)
#                 count_2 += 1

# train_mass_csv = np.array(equal_train_mass)

# Парсим и нормируем train матрицу
COLUMN_NAMES, normalize_train_matrix = parse_train_mass(train_mass=train_mass_csv, df=df)

normalize_test_matrix = parse_test_answers(train_mass=train_mass_csv)

print(normalize_train_matrix.shape)
print(normalize_test_matrix.shape)
print(normalize_test_matrix)

{'Содержание': 0, 'Сервис': 1, 'Приоритет': 2, 'Статус': 3, 'Функциональная группа': 4, 'Крайний срок': 5, 'Дата обращения': 6, 'Тип обращения на момент подачи': 7, 'Тип обращения итоговый': 8, 'Решение': 9, 'Тип переклассификации': 10, 'Дата восстановления': 11, 'Дата закрытия обращения': 12, 'Критичность': 13, 'Влияние': 14, 'Система': 15, 'Место': 16}
(14803, 5)
(14803,)
[0.33333333 0.33333333 0.33333333 ... 0.33333333 0.33333333 0.33333333]


In [74]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2(precisionrecall)/(precision+recall+K.epsilon())
    return f1_val
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

def getModel():
    model = keras.Sequential([
        tf.keras.layers.Input(shape=len(normalize_train_matrix[0])),
        tf.keras.layers.Dense(512, activation="sigmoid"),
        tf.keras.layers.Dense(256, activation="sigmoid"),
        tf.keras.layers.Dense(128, activation="sigmoid"),
        tf.keras.layers.Dense(64, activation="sigmoid"),
        tf.keras.layers.Dense(1, activation="tanh")
    ])

    model.compile(optimizer=tf.optimizers.Adam(), loss="mse", metrics=[f1_metric])

    return model

SyntaxError: invalid syntax (3253931711.py, line 1)

In [76]:
tf.keras.utils.plot_model(RNN_model, "multi_input_and_output_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [72]:
RNN_model = getModel()

In [73]:
RNN_model.fit(normalize_train_matrix, normalize_test_matrix, batch_size=32, epochs=100)

Epoch 1/100


ValueError: in user code:

    File "C:\Users\user\VUZ\HackPython\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\VUZ\HackPython\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\VUZ\HackPython\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\VUZ\HackPython\venv\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\user\VUZ\HackPython\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\user\VUZ\HackPython\venv\lib\site-packages\keras\engine\input_spec.py", line 235, in assert_input_compatibili

In [59]:
for row in normalize_test_matrix:
    print(row)

0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333

In [65]:
predicts = RNN_model.predict(normalize_train_matrix)

463/463 [==============================] - 0s 760us/step


In [66]:
for i in range(0, len(normalize_test_matrix)):
    print(predicts[i], end=f"   {normalize_test_matrix[i]}\n")

[0.33938178]   0.3333333333333333
[0.33938178]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.58959717]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.3386671]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3502355]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33372846]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.333333333

[0.33092606]   0.3333333333333333
[0.330926]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33133134]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33133134]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.8687858]   1.0
[0.330926]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.333245]   0.3333333333333333
[0.3366

[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.3377835]   0.3333333333333333
[0.3377835]   0.3333333333333333
[0.33778346]   0.

[0.33194947]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.33053064]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.33133134]   0.3333333333

[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663964]   0.3333333333333333
[0.33663964]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.33194947]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.33663973]   0.3333333333333333
[0.5927107]   0.3333333333333333
[0.5927107]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.5927107]   0.3333333333333333
[0.33778346]   0.3333333333333333
[0.3313596]   0.3333333333333333
[0.3313596]   0.33

In [36]:
from support_fucntions import parse_test_mass

In [37]:
df = pd.read_csv("dataset/test.csv")

In [38]:
test_mass_csv = np.array(df)

test_mass_without_id = test_mass_csv[:,1:]

print(test_mass_without_id[0])

COLUMN_NAMES, normalize_value_matrix = parse_test_mass(test_mass=test_mass_without_id, df=df)



['Заявка на предоставление и отзыв прав доступа к ресурсам' 'Сервис41'
 '3-Низкий' 'Закрыт' 'ФГ13' '2018-02-22 07:51' '2018-02-16 13:11' 'Запрос'
 nan 'Работы по обращению выполнены' nan '2018-02-18 04:32'
 '2018-02-25 07:56' '4-Нет' '4-Нет влияния' 'Система43' 'Место38']
{'id': 0, 'Содержание': 1, 'Сервис': 2, 'Приоритет': 3, 'Статус': 4, 'Функциональная группа': 5, 'Крайний срок': 6, 'Дата обращения': 7, 'Тип обращения на момент подачи': 8, 'Тип обращения итоговый': 9, 'Решение': 10, 'Тип переклассификации': 11, 'Дата восстановления': 12, 'Дата закрытия обращения': 13, 'Критичность': 14, 'Влияние': 15, 'Система': 16, 'Место': 17}


In [39]:
print(normalize_value_matrix.shape)

(6344, 5)


In [40]:
predicts = RNN_model.predict(normalize_value_matrix)

199/199 [==============================] - 0s 439us/step


In [41]:
len(predicts)

6344

In [47]:
df_sub = pd.read_csv("dataset/submission.csv")
print(df_sub)
for i in range(len(predicts)):
    now_type = 0
    if (predicts[i] <= 0.4):
        df_sub._set_value(i, "Тип переклассификации", "0")
    elif (predicts[i] <= 0.444444):
        df_sub._set_value(i, "Тип переклассификации", "1")
        now_type = 1
    else:
        df_sub._set_value(i, "Тип переклассификации", "2")
        now_type = 2
        
    if (now_type == 0):
        df_sub._set_value(i, "Тип обращения итоговый", "Запрос" if normalize_value_matrix[i, -2] == 1 else "Инцидент")
    elif (now_type == 1):
        df_sub._set_value(i, "Тип обращения итоговый", "Инцидент")
    else:
        df_sub._set_value(i, "Тип обращения итоговый", "Запрос")

print(df_sub)

         id  Тип переклассификации Тип обращения итоговый
0     14803                      0                 Запрос
1     14804                      0                 Запрос
2     14805                      0                 Запрос
3     14806                      0                 Запрос
4     14807                      0                 Запрос
...     ...                    ...                    ...
6339  21142                      2                 Запрос
6340  21143                      2                 Запрос
6341  21144                      2                 Запрос
6342  21145                      2                 Запрос
6343  21146                      0                 Запрос

[6344 rows x 3 columns]
         id Тип переклассификации Тип обращения итоговый
0     14803                     0                 Запрос
1     14804                     0                 Запрос
2     14805                     0                 Запрос
3     14806                     0                 З

In [45]:
print(df_sub.get("id")[0])


14803


In [46]:
df_sub.to_csv("dataset/submission.csv", index=False)